In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
etcs = gpd.read_parquet("/data/uscuni-ulce/processed_data/clusters/66593_clusters.pq")

In [ ]:
subset = etcs["final"].isin([6, 14, 5, 15, 17])

In [ ]:
import lonboard

layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=0.7)

from sidecar import Sidecar

sc = Sidecar(title="Final Clusters")
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

In [ ]:
from palettable.colorbrewer.qualitative import Set3_12
from lonboard.colormap import apply_categorical_cmap


def get_color(labels_to_color, noise_color=[0, 0, 0]):
    """Generate n colors for n labels. Labels with -1 are black."""
    import glasbey

    def hex_to_rgb(hexa):
        return tuple(int(hexa[i : i + 2], 16) for i in (0, 2, 4))

    if labels_to_color.max() >= 11:
        gb_cols = glasbey.extend_palette(
            Set3_12.hex_colors, palette_size=np.unique(labels_to_color).shape[0] + 1
        )
    else:
        gb_cols = Set3_12.hex_colors

    gb_cols = [hex_to_rgb(c[1:]) for c in gb_cols]

    colors = apply_categorical_cmap(
        labels_to_color,
        cmap=dict(zip(np.unique(labels_to_color), gb_cols, strict=False)),
    )
    colors[labels_to_color == -1] = noise_color
    return colors

In [ ]:
all_colors = get_color(labels_to_color=etcs.final)
unique_clusters = np.array(np.arange(-1, 34))
all_colors = get_color(labels_to_color=unique_clusters)
color_df = pd.DataFrame(
    data={"final_without_noise": unique_clusters, "color": all_colors.tolist()}
)
layer.get_fill_color = np.array(
    etcs.merge(color_df, on="final_without_noise", how="left")["color"].tolist(),
    dtype="uint8",
)

In [ ]:
import pandas as pd

fdata = pd.read_parquet("/data/uscuni-ulce/processed_data/clusters/fdata.pq")

In [ ]:
fdata.loc[[15, 16]].style.background_gradient(axis=1, cmap="BuGn")

In [ ]:
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)

In [ ]:
region_hulls.explore()